# BANDGAP_1v_v01
A bandgap in the SKY130 process.  Outputs a 1v reference voltage.

## Table of Contents
- [Introduction and Setup](#introduction)
- [DC Operating Point Test](#dcop_test)
- [Setup Appendix](#setup-appendix)

## Setup
A complete analog and digital open-source IC design environment can be ran inside a Docker 
devcontainer based on the [iic-osic-tools](https://github.com/iic-jku/iic-osic-tools) image from JKU University.  Then  the repo can be opened in vscode directly using the remote devcontainer extension.
This will turn your vscode session into a full analog design environemnt with 
no additional setup required. It will include all the simulators, schematic editors, layout editors, and 
software tools necessary for both analog and digital IC design.

For more details on the environemnt setup see the [Setup Appendix](#setup-appendix) cell.

## Notebook Setup
Initializes some imports and paths that will be needed by the rest of the notebooks.

<a id='introduction'></a>

In [1]:
! source ~/.bashrc
import os
from pathlib import Path
from rich.jupyter import print
import ipytest

from viper.simulators.TestSimulator import TestSimulator

DEBUG = True
BANDGAP_REPO_ROOT = Path("/workspaces/prjs/bandgapReferenceCircuit")
TESTS_DIR_PATH = BANDGAP_REPO_ROOT / "tests"
SCHEMATIC_DIR_PATH = BANDGAP_REPO_ROOT / "bandgap_sky130_v1"

ipytest.autoconfig()

# Check environment setup
if not Path(os.environ["WORKSPACE_DIR"]).exists():
    raise Exception("Workspace directory path needs to be updated in the .env"
                    " file.")

if DEBUG:
    print("$PATH = ")
    print(os.environ["PATH"].split(":"))
    WORKSPACE_DIR = os.environ["WORKSPACE_DIR"]
    print(f"WORKSPACE_DIR = {WORKSPACE_DIR}")
    WORKSPACE_DIR = os.environ["SPICE_ASCIIRAWFILE"]
    print(f"SPICE_ASCIIRAWFILE = {WORKSPACE_DIR}")
    if False:
        print("All ENV VARS:")
        print(os.environ)

[INFO] Final PATH variable: /foss/tools/bin:/foss/tools/sak/:/usr/local/sbin:/opt/conda/envs/bandgap/bin:/opt/conda/condabin:/foss/tools/xschem/bin:/foss/tools/ngspice/ngspice/bin:/vscode/vscode-server/bin/linux-x64/7f329fe6c66b0f86ae1574c2911b681ad5a45d63/bin/remote-cli:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/foss/tools/covered/19d30fc/bin:/foss/tools/cvc_rv/df85a63/bin:/foss/tools/fault/080f4be/:/foss/tools/gaw3-xschem/a4bb956/bin:/foss/tools/gds3d/173da0c/bin:/foss/tools/ghdl/v3.0.0/bin:/foss/tools/gtkwave/816166e/bin:/foss/tools/iic-osic/:/foss/tools/irsim/25fe821/bin:/foss/tools/iverilog/0144168/bin:/foss/tools/klayout/44a2aa9/:/foss/tools/magic/5e5879c/bin:/foss/tools/netgen/28a2950/bin:/foss/tools/ngspice/ngspice/bin:/foss/tools/nvc/r1.8.2/bin:/foss/tools/openlane/2023.03/:/foss/tools/openroad/1a1617d/bin:/foss/tools/opensta/489ffac/bin:/foss/tools/padring/b2a64ab/bin:/foss/tools/qflow/b0f76bf/bin:/foss/tools/verilator/v5.008/bin:/foss/tools/

$PATH =

[
    '/opt/conda/envs/bandgap/bin',
    '/opt/conda/condabin',
    '/foss/tools/xschem/bin',
    '/foss/tools/ngspice/ngspice/bin',
    '/vscode/vscode-server/bin/linux-x64/7f329fe6c66b0f86ae1574c2911b681ad5a45d63/bin/remote-cli',
    '/opt/conda/bin',
    '/usr/local/sbin',
    '/usr/local/bin',
    '/usr/sbin',
    '/usr/bin',
    '/sbin',
    '/bin'
]

WORKSPACE_DIR = /workspaces/prjs/bandgapReferenceCircuit

SPICE_ASCIIRAWFILE = 1

<a id='dcop_test'></a>

# DC Operating Point Test

The DC operating point test checks the DC output voltage of the bandgap.

It also checks the terminal voltages (Vds, Vgs, Vgd) of each transistor are 
within the safe operating area. (SOA)  This helps ensure they operate 
reliably over time.

## Run DC Op Simulation

In [2]:

test_name = "dc_op"
test_bench_schematic = "bandgap_1v_v01_dcop_testbench.sch"

dcop_result = TestSimulator.run(
    schematic_filepath=SCHEMATIC_DIR_PATH / test_bench_schematic, 
    test_dirpath=TESTS_DIR_PATH / test_name,
    )
print(dcop_result)

Netlisting bandgap_1v_v01_dcop_testbench.sch
  From schematic path: /workspaces/prjs/bandgapReferenceCircuit/bandgap_sky130_v1/bandgap_1v_v01_dcop_testbench.sch
  To netlist file: bandgap_1v_v01_dcop_testbench.spice
  To netlist path: /workspaces/prjs/bandgapReferenceCircuit/tests/dc_op/netlist/bandgap_1v_v01_dcop_testbench.spice
  Netlisting complete!

Simulating bandgap_1v_v01_dcop_testbench.spice at 
  /workspaces/prjs/bandgapReferenceCircuit/tests/dc_op/netlist/bandgap_1v_v01_dcop_testbench.spice
  using ngspice
  Simulation completed!


Simulation Result Details:
  simulator: ngspice
  directory path: /workspaces/prjs/bandgapReferenceCircuit/tests/dc_op/simulation
  output file: bandgap_1v_v01_dcop_testbench.out
  raw output file: bandgap_1v_v01_dcop_testbench.raw
  SOA log file: bandgap_1v_v01_dcop_testbench.soa.log
  sim log file: bandgap_1v_v01_dcop_testbench.sim.log
  netlist filepath: 
/workspaces/prjs/bandgapReferenceCircuit/tests/dc_op/netlist/bandgap_1v_v01_dcop_testbench.spice

## Check DC Op Results

In [3]:
%%ipytest -qq
# %load tests/dc_op/test_op_point.py
import pytest
from pathlib import Path
from spyci import spyci
from rawread import rawread

from viper.testing.PerformanceTest import PerformanceTest
from viper.simulators.SimResult import SimResult

@pytest.fixture(scope="module")
def performance_test(request: pytest.FixtureRequest) -> PerformanceTest:

    try:
        return PerformanceTest.read_package(request.module.__package__)
    except:
        return PerformanceTest.read_directory(
            "/workspaces/prjs/bandgapReferenceCircuit/tests/dc_op"
        )

@pytest.fixture(scope="module")
def sim_result(performance_test: PerformanceTest) -> Path:
    return performance_test.sim_result

def test_result_complete(sim_result: SimResult):
    print(f"output_filepath: {sim_result.output_filepath}")
    assert sim_result.output_filepath.exists()
    assert sim_result.raw_output_filepath.exists()

# def test_()

#def test_vdsat(dcop_result):
    # data = spyci.load_raw(str(dcop_result))
    #data = rawread(str(dcop_result))
    #assert isinstance(dcop_result, ngspice_result)
 #   assert 1 < 2
    #assert V_bg < 1.05


# if __name__ == "__main__":
#     test_vdsat(Path("/workspaces/bandgapReferenceCircuit/tests/dc_op/simulation/tsmc_bandgap_real_op.raw"))

output_filepath: /workspaces/prjs/bandgapReferenceCircuit/tests/dc_op/simulation/bandgap_1v_v01_dcop_testbench.out

.                                                                                            [100%]


## DC Op Performance
Reports the DC operating point performance of the bandgap

<a id='setup_appendix'></a>

# Setup Appendix
This section provides more detail of our EDA setup.

### Jupyter setup
Our jupyter session requires the .env to include our EDA tools on the $PATH since the .bashrc isn't ran as part of the Jupyter Kernel's environment.